In [ ]:
# sentence-transformers 설치
!pip install -q sentence-transformers

# tqdm (진행률 표시용)
!pip install -q tqdm

In [ ]:
import json
import torch
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import os

# 1️⃣ GPU 사용 확인
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# 2️⃣ 모델 불러오기
model = SentenceTransformer("nlpai-lab/KURE-v1", device=device)

# 3️⃣ 파일 경로 설정 (Google Drive 연동 시 경로 변경 가능)
input_file = "/content/chunked_label.jsonl"  # 실제 파일 경로
output_file = "/content/chunked_label_embedded.jsonl"

# 4️⃣ 배치 사이즈 설정 (GPU 권장: 128~256)
batch_size = 128

# 5️⃣ 중간 저장 단위 (몇 개씩 파일에 쓰면서 저장할지)
save_every = 1024  # 배치 수 기준, 필요시 조절

# 6️⃣ 임베딩 진행
batch_texts, batch_records = [], []
total_processed = 0

# UTF-8 깨짐 방지
with open(input_file, "r", encoding="utf-8", errors="ignore") as infile, \
     open(output_file, "w", encoding="utf-8") as outfile:

    # 전체 라인 수 계산 (진행바용)
    total_lines = sum(1 for _ in open(input_file, "r", encoding="utf-8", errors="ignore"))
    infile.seek(0)

    for line in tqdm(infile, total=total_lines, desc="Embedding"):
        record = json.loads(line.strip())
        batch_texts.append(record.get("answer_text", ""))
        batch_records.append(record)

        # 배치 단위로 임베딩
        if len(batch_texts) == batch_size:
            embeddings = model.encode(batch_texts, convert_to_numpy=True)
            for rec, emb in zip(batch_records, embeddings):
                rec["embedding"] = emb.tolist()
                outfile.write(json.dumps(rec, ensure_ascii=False) + "\n")

            total_processed += len(batch_texts)
            batch_texts, batch_records = [], []

            # 중간 진행 상황 출력
            if total_processed % (batch_size * save_every) == 0:
                print(f"💾 {total_processed} records embedded and saved...")

    # 마지막 남은 배치 처리
    if batch_texts:
        embeddings = model.encode(batch_texts, convert_to_numpy=True)
        for rec, emb in zip(batch_records, embeddings):
            rec["embedding"] = emb.tolist()
            outfile.write(json.dumps(rec, ensure_ascii=False) + "\n")
        total_processed += len(batch_texts)

print(f"✅ 임베딩 완료! 총 {total_processed}개 레코드 처리. 결과 파일: {output_file}")